In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
import torch
from transformers import  AutoTokenizer
from transformers import AutoModelForCausalLM, AutoConfig

model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.bos_token = tokenizer.eos_token

# config = AutoConfig.from_pretrained(
#     model_name,
#     vocab_size=len(tokenizer),
#     bos_token_id=tokenizer.bos_token_id,
#     eos_token_id=tokenizer.eos_token_id,
#     trust_remote_code=True
# )

# phi2_model_pretrained = AutoModelForCausalLM.from_config(config, 
#                                                          trust_remote_code=True, 
#                                                          )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model_name = "microsoft/phi-2"
phi2_model_pretrained = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,  
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
special_tokens_dict = {'pad_token': '<|PAD|>', 'bos_token': '<|BOS|>'}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print('We have added', num_added_toks, 'tokens')
phi2_model_pretrained.resize_token_embeddings(len(tokenizer))  # Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e. the length of the tokenizer.

We have added 2 tokens


Embedding(50297, 2560)

### Create dataset

In [5]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd 
import json
import os 
import h5py

In [6]:
def get_image_name(image_id_from_caption, list_image_info): 
    for img in list_image_info: 
        if img['id'] == image_id_from_caption: 
            img_name = img['file_name'].split('.')[0]
            return img['file_name'].split('.')[0]
    return 'NoImgNameFound'

In [7]:
# file_path_captions_coco = '/media/App/amaranth/lavanya/Capstone_data/annotations_trainval2017/annotations/captions_train2017.json'

# with open(file_path_captions_coco) as f:
#    data = json.load(f)

# captions_info = []
# for a in data['annotations']: 
#     captions_info.append([a['image_id'], a['caption'], a['id']])

# captions_info_df = pd.DataFrame(data=captions_info, columns=['image_id', 'caption', 'caption_id'])
# captions_info_df['image_name'] = captions_info_df['image_id'].apply(lambda x: get_image_name(x, data['images']))
# captions_info_df['image_name'] = captions_info_df['image_name'].apply(lambda x: '0'*(12-len(str(x))) + str(x))
# captions_info_df.to_csv('captions_images_map_COCO_train2017.csv', index = False)

In [8]:
captions_info_df = pd.read_csv('captions_images_map_COCO_train2017.csv')

/tmp/ipykernel_390507/1237926302.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  captions_info_df = pd.read_csv('captions_images_map_COCO_train2017.csv')


In [9]:
import h5py    
import numpy as np    

In [10]:
class COCO_CLIP_Dataset(Dataset):

    def __init__(
        self, caption_file, embedding_path, tokenizer, max_token_len_data):
        
        self.embedding_path = embedding_path
        self.caption_file = caption_file
        self.tokenizer = tokenizer
        self.max_token_len_data = max_token_len_data

    def __len__(self):
        return len(self.caption_file)
    
    def __getitem__(self, index):

        row = self.caption_file.iloc[[index]]

        df_img = row['image_id'].values[0]
        img_base_name = '0'*(12-len(str(df_img))) + str(df_img)
        img_base_name = img_base_name.replace(' ', '0')
        img_clip_embedding_path = os.path.join(self.embedding_path, f'{img_base_name}.h5')

        np_array_embed_img = h5py.File(img_clip_embedding_path,'r+')['image_features'][()]
        
        img_caption = row['caption'].values[0] ## Tokenize this 
        img_caption_tokenized = self.tokenizer(img_caption, return_tensors="pt", 
                                               return_attention_mask=False).input_ids

        ## put bos, eos, and padding for batch         
        input_bos = torch.cat((torch.tensor(self.tokenizer.bos_token_id).view((1,1)), 
                                                       img_caption_tokenized), dim=1)

        input_bos = img_caption_tokenized

        input_eos = torch.cat((input_bos, 
                               torch.tensor(self.tokenizer.eos_token_id).view((1,1))), dim=1)
        
        if (self.max_token_len_data - input_eos.shape[1]) > 0: 
            input_final =  torch.cat((input_eos,torch.tensor([self.tokenizer.pad_token_id]*(self.max_token_len_data - input_eos.shape[1])).unsqueeze(0)), dim=1)
        else: 
            input_final = input_eos
        
        return torch.tensor(np_array_embed_img).squeeze(0), input_final.squeeze(0)

In [11]:
def file_exists(image_id, fpath = '/media/App/amaranth/lavanya/Capstone_data/clip_features_base_patch32/'): 

    n = '0'*(12-len(str(image_id))) + str(image_id) + '.h5'
    fp = os.path.join(fpath, n)

    if os.path.exists(fp): 
        return True
    else: 
        return False

In [12]:
### captions_info_df contains for 1 image multiple entries, lets reduce keeping one image, one entry. 
captions_info_df_subset = captions_info_df.drop_duplicates(subset='image_id', keep='first')
captions_info_df_subset['image_embed_exists'] = captions_info_df_subset['image_id'].apply(lambda x: file_exists(x))

/tmp/ipykernel_390507/1249603411.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  captions_info_df_subset['image_embed_exists'] = captions_info_df_subset['image_id'].apply(lambda x: file_exists(x))


In [13]:
len(captions_info_df_subset), len(captions_info_df_subset[captions_info_df_subset['image_embed_exists'] == True])

(118287, 118287)

In [14]:
max_token_len_data = 75
dataset = COCO_CLIP_Dataset(captions_info_df_subset, 
                            '/media/App/amaranth/lavanya/Capstone_data/clip_features_base_patch32/', 
                            tokenizer, max_token_len_data)

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleResBlock(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.pre_norm = nn.LayerNorm(input_size)
        self.proj = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.GELU(),
            nn.Linear(input_size, input_size)
        )
    def forward(self, x):
        x = self.pre_norm(x)
        return x + self.proj(x)
    
class Phi2wrapper(nn.Module):
    
    #This defines the structure of the NN.
    def __init__(self, input_dim_CLIP=768, input_dim_phi2=2560, 
                 phi2_model=phi2_model_pretrained, 
                 max_token_len_data=max_token_len_data, tokenizer=tokenizer):
        
        super(Phi2wrapper, self).__init__()

        self.input_dim_CLIP = input_dim_CLIP
        self.input_dim_phi2 = input_dim_phi2
        self.projection_img = nn.Linear(self.input_dim_CLIP, self.input_dim_phi2, 
                                        bias=False)
        self.resblock = SimpleResBlock(self.input_dim_phi2)
        self.phi2_model = phi2_model
        self.max_token_len_data = max_token_len_data
        self.tokenizer = tokenizer

        self.device = device
        self.bos_embedding  = self.phi2_model.get_input_embeddings()(torch.tensor(self.tokenizer.bos_token_id).to(self.device)).unsqueeze(0)

        eoi = self.tokenizer(". Caption this image.", 
                return_tensors="pt", return_attention_mask=False)
        self.eoi_embedding = self.phi2_model.get_input_embeddings()(eoi.input_ids.to(self.device)).squeeze(0)

    def forward(self, x, caption):

        x = self.projection_img(x)
        x = self.resblock(x)

        batch_size = x.shape[0]

        x = torch.cat((bos_embedding.repeat(batch_size,1,1), x, eoi_embedding.repeat(batch_size,1,1)), dim=1)
        
        loss = 0 
        word_output_pred_tokens = torch.empty((batch_size, input_caption.shape[1]))

        for idx in range(input_caption.shape[1]): 
            
            next_word = phi2_model_pretrained.generate(inputs_embeds=x, max_new_tokens = 1, output_scores=True, return_dict_in_generate = True, 
                                            pad_token_id=tokenizer.pad_token_id, 
                                            bos_token_id=tokenizer.bos_token_id, 
                                            eos_token_id=tokenizer.eos_token_id) ## this gives first word  
            caption_word_token = input_caption[:,idx]

            if sum(torch.eq(torch.tensor([tokenizer.pad_token_id]*batch_size), caption_word_token)) == torch.tensor(batch_size): 
                break 

            caption_word_embedding = phi2_model_pretrained.get_input_embeddings()(caption_word_token.to(device)).unsqueeze(1)
            x = torch.cat((x, caption_word_embedding), dim=1)

            loss += F.cross_entropy(next_word.scores[0], caption_word_token.to(device), 
                        ignore_index=tokenizer.pad_token_id, label_smoothing=0.1)

            word_output_pred_tokens[:, idx] = next_word.sequences[:, 1]

            loss_tosend = loss/batch_size

        return loss_tosend, word_output_pred_tokens

        ### Without feature forcing
        # x = self.phi2_model.generate(inputs_embeds=x, 
        #                              max_new_tokens=self.max_token_len_data, 
        #                              output_scores=True, return_dict_in_generate = True, 
        #                              pad_token_id=self.tokenizer.eos_token_id, 
        #                              bos_token_id=self.tokenizer.bos_token_id, 
        #                              eos_token_id=self.tokenizer.eos_token_id)

        # return x 

device = 'cuda:1'
torch.set_grad_enabled(True)  
phi2_projection_model = Phi2wrapper().to(device=device)

## Freezing phi-2 for projection layer training 
for name, param in phi2_projection_model.named_parameters():
    if "phi2_model" in name:
        param.requires_grad = False
    else: 
        param.requires_grad = True

In [293]:
batch_size_train = 4
train_dataloader = DataLoader(dataset, batch_size=batch_size_train, shuffle=True)

In [294]:
input_img_embed, input_caption = next(iter(train_dataloader))

In [295]:
input_img_embed.shape, input_caption.shape

(torch.Size([4, 49, 768]), torch.Size([4, 75]))

In [296]:
x = phi2_projection_model(input_img_embed.to(device))
x.shape

torch.Size([4, 49, 2560])

In [297]:
bos_embedding = phi2_model_pretrained.get_input_embeddings()(torch.tensor(tokenizer.bos_token_id).to(device)).unsqueeze(0)
eos_embedding = phi2_model_pretrained.get_input_embeddings()(torch.tensor(tokenizer.eos_token_id).to(device)).unsqueeze(0)
pad_embedding = phi2_model_pretrained.get_input_embeddings()(torch.tensor(tokenizer.pad_token_id).to(device)).unsqueeze(0)
eoi = tokenizer(". Caption this image.", return_tensors="pt", return_attention_mask=False)
eoi_embedding = phi2_model_pretrained.get_input_embeddings()(eoi.input_ids.to(device)).squeeze(0)

In [298]:
eoi_embedding.shape, bos_embedding.shape

(torch.Size([5, 2560]), torch.Size([1, 2560]))

In [299]:
eoi_embedding.repeat(4,1,1).shape, bos_embedding.repeat(4,1,1).shape

(torch.Size([4, 5, 2560]), torch.Size([4, 1, 2560]))

In [300]:
batch_size = x.shape[0]
batch_size 

4

In [301]:
x = torch.cat((bos_embedding.repeat(batch_size,1,1), x, eoi_embedding.repeat(batch_size,1,1)), dim=1)

In [302]:
x.shape

torch.Size([4, 55, 2560])

In [303]:
input_caption.shape

torch.Size([4, 75])

In [304]:
loss = 0 
word_output_pred_tokens = torch.empty((batch_size, input_caption.shape[1]))
loss_to_divide = 0
for idx in range(input_caption.shape[1]): 
    
    next_word = phi2_model_pretrained.generate(inputs_embeds=x, max_new_tokens = 1, output_scores=True, return_dict_in_generate = True, 
                                     pad_token_id=tokenizer.pad_token_id, 
                                     bos_token_id=tokenizer.bos_token_id, 
                                     eos_token_id=tokenizer.eos_token_id) ## this gives first word  
    caption_word_token = input_caption[:,idx]
    no_of_pad_tokens = sum(torch.eq(torch.tensor([tokenizer.pad_token_id]*batch_size), caption_word_token))
    if no_of_pad_tokens == torch.tensor(batch_size): 
        break 
    
    caption_word_embedding = phi2_model_pretrained.get_input_embeddings()(caption_word_token.to(device)).unsqueeze(1)
    x = torch.cat((x, caption_word_embedding), dim=1)

    loss += F.cross_entropy(next_word.scores[0], caption_word_token.to(device), 
                ignore_index=tokenizer.pad_token_id, label_smoothing=0.1)
    print(loss, caption_word_token, next_word.sequences[:, 1], no_of_pad_tokens)
    loss_to_divide += batch_size - no_of_pad_tokens
    word_output_pred_tokens[:, idx] = next_word.sequences[:, 1]

loss_tosend = loss/loss_to_divide

tensor(10.2231, device='cuda:1') tensor([32, 32, 32, 32]) tensor([198, 198, 198, 198], device='cuda:1') tensor(0)
tensor(17.9497, device='cuda:1') tensor([11849,  1448,  3155, 13791]) tensor([649, 649, 649, 649], device='cuda:1') tensor(0)
tensor(23.0475, device='cuda:1') tensor([ 1323,   286,   286, 19401]) tensor([4512,  286,  286,   12], device='cuda:1') tensor(0)
tensor(29.9448, device='cuda:1') tensor([ 1474,   661, 37370,   351]) tensor([  287,   661,  1528, 17607], device='cuda:1') tensor(0)
tensor(34.0637, device='cuda:1') tensor([  257,  5055, 21223,   257]) tensor([ 262,  287, 2001,  257], device='cuda:1') tensor(0)
tensor(40.7156, device='cuda:1') tensor([1323, 1088, 5055, 6473]) tensor([5103,  287,   12, 2266], device='cuda:1') tensor(0)
tensor(45.0068, device='cuda:1') tensor([2245, 4964, 1306,  338]) tensor([2245,  257,  287, 4397], device='cuda:1') tensor(0)
tensor(49.2858, device='cuda:1') tensor([  351, 26056,   284,  4151]) tensor([287, 257, 284,  12], device='cuda:1'

In [290]:
loss_to_divide

tensor(53)

In [292]:
loss

tensor(79.6454, device='cuda:1')

In [291]:
loss_tosend

tensor(1.5027, device='cuda:1')

In [263]:
word_output_pred_tokens.shape

torch.Size([4, 75])

In [164]:
word_output_pred_tokens = torch.empty((batch_size, input_caption.shape[1]))
word_output_pred_tokens.shape

torch.Size([4, 75])

In [163]:
next_word.sequences[:, 1].unsqueeze(1).shape

torch.Size([4, 1])

In [156]:
next_word.scores[0]

torch.Size([4, 50297])

In [107]:
## DEBUG Feature-forcing
input_img_embed = dataset[0][0]
input_caption = dataset[0][1]

x = phi2_projection_model(input_img_embed.to(device))
x.shape

torch.Size([49, 2560])

In [108]:
bos_embedding = phi2_model_pretrained.get_input_embeddings()(torch.tensor(tokenizer.bos_token_id).to(device)).unsqueeze(0)
eos_embedding = phi2_model_pretrained.get_input_embeddings()(torch.tensor(tokenizer.eos_token_id).to(device)).unsqueeze(0)
pad_embedding = phi2_model_pretrained.get_input_embeddings()(torch.tensor(tokenizer.pad_token_id).to(device)).unsqueeze(0)
eoi = tokenizer(". Caption this image.", return_tensors="pt", return_attention_mask=False)
eoi_embedding = phi2_model_pretrained.get_input_embeddings()(eoi.input_ids.to(device)).squeeze(0)

In [109]:
eoi_embedding.shape, bos_embedding.shape

(torch.Size([5, 2560]), torch.Size([1, 2560]))

In [110]:
x = torch.cat((bos_embedding, x, eoi_embedding), dim=0).unsqueeze(0)
x.shape

torch.Size([1, 55, 2560])

In [111]:
next_word.sequences.shape

torch.Size([1, 2])

In [113]:
input_caption

tensor([   32, 17026, 30069,   351,   257,  8801,   355,   262,  2166,  7825,
           13, 50256, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295,
        50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295,
        50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295,
        50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295,
        50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295,
        50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295, 50295,
        50295, 50295, 50295, 50295, 50295])

In [112]:
loss = 0 
word_output_pred_tokens = []

for idx, caption_word_token in enumerate(input_caption): 
    
    next_word = phi2_model_pretrained.generate(inputs_embeds=x, max_new_tokens = 1, output_scores=True, return_dict_in_generate = True, 
                                     pad_token_id=tokenizer.eos_token_id, 
                                     bos_token_id=tokenizer.bos_token_id, 
                                     eos_token_id=tokenizer.eos_token_id) ## this gives first word  
    
    caption_word_embedding = phi2_model_pretrained.get_input_embeddings()(caption_word_token.to(device)).unsqueeze(0).unsqueeze(0)
    x = torch.cat((x, caption_word_embedding), dim=1)

    loss += F.cross_entropy(next_word.scores[0].squeeze(0), caption_word_token.to(device), ignore_index=tokenizer.pad_token_id, label_smoothing=0.1)

    word_output_pred = next_word.sequences[0][1]
    word_output_pred_tokens.append(word_output_pred)

loss_tosend = loss/len(input_caption)
tokenizer.batch_decode(word_output_pred_tokens)

shape of x torch.Size([1, 55, 2560])
shape of x torch.Size([1, 56, 2560])
shape of x torch.Size([1, 57, 2560])
shape of x torch.Size([1, 58, 2560])
shape of x torch.Size([1, 59, 2560])
shape of x torch.Size([1, 60, 2560])
shape of x torch.Size([1, 61, 2560])
shape of x torch.Size([1, 62, 2560])
shape of x torch.Size([1, 63, 2560])
shape of x torch.Size([1, 64, 2560])
shape of x torch.Size([1, 65, 2560])
shape of x torch.Size([1, 66, 2560])
shape of x torch.Size([1, 67, 2560])
shape of x torch.Size([1, 68, 2560])
shape of x torch.Size([1, 69, 2560])
shape of x torch.Size([1, 70, 2560])
shape of x torch.Size([1, 71, 2560])
shape of x torch.Size([1, 72, 2560])
shape of x torch.Size([1, 73, 2560])
shape of x torch.Size([1, 74, 2560])
shape of x torch.Size([1, 75, 2560])
shape of x torch.Size([1, 76, 2560])
shape of x torch.Size([1, 77, 2560])
shape of x torch.Size([1, 78, 2560])
shape of x torch.Size([1, 79, 2560])
shape of x torch.Size([1, 80, 2560])
shape of x torch.Size([1, 81, 2560])
s

['\n',
 ' new',
 ' is',
 ' of',
 ' a',
 ' red',
 'work',
 ' a',
 ' backdrop',
 ' wheel',
 '.',
 '\n',
 'The',
 ' and',
 ':',
 '.',
 '\n',
 ':',
 ':',
 ':',
 '\n',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 'about',
 'about',
 'about',
 'about',
 'about',
 'about',
 'about',
 'about',
 'about',
 'about',
 'our',
 'our',
 'our',
 'our',
 'our',
 'our',
 'our',
 'our',
 'our',
 'our',
 'our',
 'our',
 'our',
 'our',
 'our',
 'all',
 'our',
 'our']

tensor(14.3412, device='cuda:1')

## Rest is the same. 

In [19]:
batch_size_train = 60 
train_dataloader = DataLoader(dataset, batch_size=batch_size_train, shuffle=True)
num_batches_train_on = 1500    
num_batches_train_on, len(train_dataloader)

(1500, 1972)

In [20]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, 
                                    phi2_projection_model.parameters()), 
                            lr=1e-5, eps=1e-9) 

In [22]:
num_epochs = 10
vocab_size = 51200

phi2_projection_model.train()
N_batches = len(train_dataloader)
                
for epoch in range(num_epochs):

    print(f"Working on epoch {epoch}")

    for iteration, batch in enumerate(train_dataloader):

        if iteration == num_batches_train_on: 
            break 

        print(f"Iteration {iteration}/{num_batches_train_on}", end='\r')

        optimizer.zero_grad()

        input = batch[0]
        gt = batch[1] 

        output = phi2_projection_model(input.to(device))

        ## need to map gt token_ids to one-hot enocding vocab_size
        gt_one_hot = torch.nn.functional.one_hot(gt, vocab_size).to(torch.float32)

        ## output in correct shape
        output_tensor_new = torch.empty(batch_size_train, max_token_len_data, vocab_size)
        for idx, s in enumerate(output.scores): 
            output_tensor_new[:, idx, :] = s
        
        output_tensor_new = F.softmax(output_tensor_new, dim=-1)
        
        ## ce loss between output_tensor_new and gt_one_hot
        loss = F.cross_entropy(output_tensor_new.view(-1, vocab_size), gt.view(-1))

        loss.requires_grad = True
        loss.backward()

        optimizer.step() 
        # optimizer.zero_grad(set_to_none=True) 

        if (iteration % 100) == 0: 
            print("")

            ## print gt and output decoded tokens for visual inspection for the 1st el of batch
            gt_input_ids = gt[0]
            output_input_ids = torch.argmax(output_tensor_new[0], dim=1)
            output_sequences_input_ids = output.sequences[0, :]

            gt_idx_0_decoded = tokenizer.decode(gt_input_ids).replace('<|endoftext|>', '')
            output_idx_0_decoded = tokenizer.decode(output_input_ids).replace('<|endoftext|>', '')
            output_sequences_idx_0_decoded = tokenizer.decode(output_sequences_input_ids).replace('<|endoftext|>', '')

            # print(f"Loss: {loss}\nInput_ids (gt): {gt_input_ids}\nInput_ids (pred): {output_input_ids}\nInput_ids (sequence): {output_sequences_input_ids}")
            ## print loss 
            print(f"Loss: {loss}\nCaption (gt): {gt_idx_0_decoded}\nCaption (pred): {output_idx_0_decoded}\nCaption (sequence): {output_sequences_idx_0_decoded}")
 
        
    print("")
    print(f"Epoch {epoch} finished")
    print("")

Working on epoch 0
Iteration 0/1500
Loss: 10.840618133544922
Caption (gt): A man riding a snowboard down a snow covered slope.
Caption (pred): us, a former member of the House of Representatives, was a member of the Democratic Party.

In the end, the people of the United States have the power to choose their leaders through elections. This is a fundamental right that is protected by the Constitution. It is important for citizens to stay informed and participate in the election process to ensure that their voices are heard.
Caption (sequence): us, a former member of the House of Representatives, was a member of the Democratic Party.

In the end, the people of the United States have the power to choose their leaders through elections. This is a fundamental right that is protected by the Constitution. It is important for citizens to stay informed and participate in the election process to ensure that their voices are heard.
Iteration 100/1500
Loss: 10.840950965881348
Caption (gt): A black

KeyboardInterrupt: 